In [1]:
import requests
import urllib
import time
import re
import os
from bs4 import BeautifulSoup

# 완결 총 페이지 수

In [2]:
url = 'https://m.comic.naver.com/webtoon/finish'
req_finish = requests.get(url)
soup_finish = BeautifulSoup(req_finish.text, 'lxml')
total = soup_finish.select("#ct > div.section_list_toon > div.paging_type2 > em > span")
total_num = int(re.sub('(<([^>]+)>)', '', str(total[0])))
total_num

34

# 각 페이지 별 제목, 작가, 작품 url 가져오기

In [3]:
def get_urls(num):
    url = f'https://m.comic.naver.com/webtoon/finish?sort=UPDATE&page={num}'
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text,'lxml')
    page = soup.select("#ct > div.section_list_toon > ul")
    
    page_content = str(page[0])
    
    titles = re.findall('<strong class="title">.*</strong>', page_content)
    authors = re.findall('<span class="author">.*</span>', page_content)
    urls = re.findall('/webtoon/list\?titleId=\d+', page_content)
    
    return titles, authors, urls

In [4]:
titles = []
authors = []
urls = []

for i in range(1, total_num + 1):
    title, author, url = get_urls(i)
    
    titles += title
    authors += author
    urls += url

In [5]:
len(titles), len(authors), len(urls)

(1005, 1005, 1005)

In [6]:
titles[0], authors[0], urls[0]

('<strong class="title">나라 구한 전생</strong>',
 '<span class="author">양키풀</span>',
 '/webtoon/list?titleId=778741')

## 제목, 작가 명 정리
- 정규식을 이용하여 HTML 태그 지우기
- 작가에서 그림 작가만 가져오기

In [7]:
titles = list(map(lambda x:re.sub('(<([^>]+)>)', '', x), titles))
authors = list(map(lambda x:re.sub('(<([^>]+)>)', '', x), authors))

In [8]:
titles[0], authors[0]

('나라 구한 전생', '양키풀')

In [9]:
def get_illustrator(author):
    if '/' in author:
        author = author.split('/')[-1].strip()

    return author

In [10]:
authors = list(map(get_illustrator, authors))

In [11]:
authors[0]

'양키풀'

# 웹툰 별 페이지에서 썸네일 이미지 정보 가져오기

In [13]:
# 19세 이상 작품의 썸네일을 가져오기 위한 네이버 로그인
from selenium import webdriver
executable_path='./webdriver/chromedriver.exe'

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path = executable_path)
driver.get("http://www.naver.com")

In [14]:
def get_thumb_img_episode(url, num):
    url = url + f'&sortOrder=ASC&page={num}'
    driver.get(url)
#     req = requests.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    index = 1
    info = soup.select(f"#ct > ul.section_episode_list > li:nth-of-type({index}) > a > div.thumbnail > img")
    
    epis = []
    urls = []
    while len(info) > 0:
        info_detail = str(info[0])
        epi_name = info_detail.split('"')[1]
        url = info_detail.split('"')[-4]
        
        epis.append(epi_name)
        urls.append(url)
        
        index += 1
        info = soup.select(f"#ct > ul.section_episode_list > li:nth-of-type({index}) > a > div.thumbnail > img")
    return epis, urls

In [15]:
epi, imgurl = get_thumb_img_episode('https://m.comic.naver.com'+urls[0], 2)
epi[0], imgurl[0]

('31화',
 'https://image-comic.pstatic.net/webtoon/778741/31/thumbnail_202x120_7089850426517107556.jpg')

In [21]:
# 403 error를 피하기 위한 조치
import urllib.request

opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

In [25]:
urllib.request.urlretrieve(imgurls[k], img_file_name)

('./data/imgs/자유/롤랑롤랑_0152.jpg', <http.client.HTTPMessage at 0x2e00e5fb948>)

In [26]:
%%time
from IPython.display import clear_output
import time

domain = 'https://m.comic.naver.com'

# 각 작품별
for i in range(len(titles)):
    
    # 폴더
    img_folder = './data/imgs/' + authors[i]
    if not os.path.exists(img_folder):
        os.mkdir(img_folder)
    
    # 작품 url 내
    url_main = domain + urls[i]
    driver.get(url_main)
#     source = requests.get(url_main)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        last = int(re.sub('(<([^>]+)>)', '', str(soup.select('#ct > div.paging_type2 > em > span')[0])))
    except:
        last = 1
    
    # 전체 페이지
    idx = 1
    for j in range(1, last+1):
        print(f'{titles[i]} - {j}/{last} Downloading...', end='\r')
        epis, imgurls = get_thumb_img_episode(url_main, j)
        
        # 각 페이지별 썸네일 이미지 저장
        for k in range(len(imgurls)):
            # 중복 방지를 위하여 페이지, 이미지 인덱스를 파일명에 추가
#             img_file_name = img_folder + '/' + titles[i] + '_' + re.sub('[^0-9a-zA-Zㄱ-힗\s]', '', epis[k]) + '_' + str(j) + '_' + str(k) + '.jpg'

            # 에피소드명 대신 회차를 파일명으로
            img_file_name = img_folder + '/' + re.sub('[:\?]', '', titles[i]) + '_' + f'{idx:04d}' + '.jpg'
            try:                
                urllib.request.urlretrieve(imgurls[k], img_file_name)
            except:
                idx += 1
                continue
            idx += 1
        
        print(f'{titles[i]} - {j}/{last} Download Complete')
        
    clear_output()

    print(f'{titles[i]} - {i+1}/{len(titles)} Complete')
clear_output()
print('All Complete')

All Complete
Wall time: 1h 9min 31s


In [86]:
driver.close()

# 연재 중 작품

In [3]:
domain_url = 'https://comic.naver.com'
main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

In [7]:
%%time

from IPython.display import clear_output

for i, each_webtoon in enumerate(webtoon_area):
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)
    title = soup.select('.comicinfo h2')[0].text.strip().split('\n')[0]
    author = get_illustrator(soup.select('.comicinfo h2')[0].text.strip().split('\n')[1].strip())
    
    img_folder = './data/imgs_week/' + author
    if not os.path.exists(img_folder):
        os.mkdir(img_folder)
    
    idx = 1
    for page_num in range(1, last_page_num + 1):
        print(f'{title} - {page_num}/{last_page_num} Downloading...', end='\r')
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]
        for each_img_tag in thumbnail_img_list:
            img_file_name = img_folder + '/' + re.sub('[:\?]', '', title) + '_' + f'{idx:04d}' + '.jpg'
            try:
                urllib.request.urlretrieve(each_img_tag['src'], img_file_name)
            except:
                idx += 1
                continue
            idx += 1
        
        print(f'{title} - {page_num}/{last_page_num} Download Complete')
        
    clear_output()

    print(f'{title} - {i+1}/{len(webtoon_area)} Complete')
clear_output()
print('All Complete')

All Complete
Wall time: 22min 34s
